In [1]:
import os
import cv2
import csv
import numpy as np
import pandas as pd
from PIL import Image
import opendatasets as od
from matplotlib import pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

In [2]:

#Read images
dir_name = 'moles-1k-images/Images/'

df = pd.read_csv('moles-1k-images/data.csv')

f = open('moles-1k-images/pixel_data.csv', 'w')
f2 = open('moles-1k-images/mole_data.csv', 'w')

bm = df.iloc[:,3:4].values

# Get list of all files in a given directory sorted by name
list_of_files = sorted( filter( lambda x: os.path.isfile(os.path.join(dir_name, x)), os.listdir(dir_name) ) )

contrast=2
imageSize=28

def functionPrint(fileName,bmData):
    
    im = cv2.resize(cv2.imread(fileName, cv2.IMREAD_GRAYSCALE), (720, 720), interpolation = cv2.INTER_AREA)
    crop = im[200:500, 200:500] 

    adjusted = cv2.convertScaleAbs(crop, alpha=contrast, beta=0) # increase contrast
    invert = cv2.bitwise_not(np.asarray(adjusted)) # black to while/ white to black
    arr = cv2.resize(invert, (imageSize, imageSize), interpolation = cv2.INTER_AREA)

    np.set_printoptions(threshold=np.inf)
    im_list = np.array(arr) 
    
    f2.write('{},\n'.format(bmData)) 
    
    for row in im_list:
        for num in row:
            f.write('{},'.format(num))

    dataIm = Image.fromarray(im_list)
    display(dataIm)

    f.write('{}\n'.format(''))
    
    
count=0
countBening=0
for file_name in list_of_files:
    
    if bm[count]=='benign':
        if countBening<159:
            functionPrint(dir_name+file_name, bm[count])
            countBening+=1
        
    elif bm[count]=='malignant':
        functionPrint(dir_name+file_name, bm[count]) 
        
    count+=1
    
f.close()
f2.close()

In [3]:
# now get classifier for this image
X = pd.read_csv('moles-1k-images/pixel_data.csv') 
y = pd.read_csv('moles-1k-images/mole_data.csv') 

In [4]:
y = y.drop(columns=y.columns[1])
X = X.drop(columns=X.columns[imageSize*imageSize])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [6]:
list_df = X.values #list of lists
y = y.values.ravel()

In [7]:
knn = KNeighborsClassifier(n_neighbors=10)

knn.fit(X, y)
knn.predict(X_test)[0:1]
knn.score(X_test,y_test)

0.703125

In [8]:
cv_scores = cross_val_score(knn, X, y, cv=10)
print(cv_scores)

[0.65625    0.53125    0.59375    0.65625    0.6875     0.5
 0.71875    0.87096774 0.74193548 0.51612903]
